In [1]:
token = {"username":"nero18","key":"d8628b574fd3cc95931821e3399a914a"}
import json 
with open('kaggle.json','w') as token_file:
  json.dump(token, token_file)

In [6]:
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download raddar/vinbigdata-competition-jpg-data-3x-downsampled

mkdir: cannot create directory ‘/home/davis20/.kaggle’: File exists
100%|█████████████████████████████████████▉| 3.39G/3.39G [01:44<00:00, 38.3MB/s]
100%|██████████████████████████████████████| 3.39G/3.39G [01:44<00:00, 34.9MB/s]


In [ ]:
!unzip vinbigdata-competition-jpg-data-3x-downsampled.zip -d data

In [8]:
import json
import os
root_dir= 'data/vinbigdata-coco-dataset-with-wbf-3x-downscaled'
with open(os.path.join(root_dir, 'train_annotations.json')) as outfile:
    data = json.load(outfile)
  

In [9]:
for x in data:
  print(x)
print(data['images'][0])

info
licenses
images
type
annotations
categories
{'license': 0, 'url': None, 'file_name': 'train_images/bf33d826094fabd938f69b3ba663f607.jpg', 'height': 1004, 'width': 1023, 'date_captured': None, 'id': 0}


In [13]:
imgs = os.listdir('data/test/test')
root_dir = 'data/test/test/'
images_dict = []
import numpy as np 
from PIL import Image 
from tqdm import tqdm 
for i, path in tqdm(enumerate(imgs)):
  img = np.array(Image.open(os.path.join(root_dir, path)))
  h, w = img.shape
  now = {
      'license': 0,
      'url': None,
      'file_name': path,
      'height': h,
      'width': w,
      'date_captured': None,
      'id': i
  } 
  images_dict.append(now)
  # break
print(len(images_dict))

3000it [00:25, 119.55it/s]

3000


In [14]:

data['images'] = images_dict 
data['annotations'] = []

In [15]:
with open('data/vinbigdata-coco-dataset-with-wbf-3x-downscaled/test_annotations.json','w') as file :
  json.dump(data, file) 

In [ ]:
!python eval_coco.py --config configs/val/test.yaml --gpus 0

In [25]:
import json
import os
with open(os.path.join('test_bbox_results.json')) as outfile:
    data = json.load(outfile)

print(len(data))

92258


In [26]:
from pycocotools.coco import COCO
coco = COCO('data/vinbigdata-coco-dataset-with-wbf-3x-downscaled/test_annotations.json')
datasets = coco.getImgIds()

print(len(datasets), coco.loadImgs(0)[0]['file_name'][5:-4])

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
3000 16bee7e6ccda97c07d096008fea


In [27]:
(np.array(data[0]['bbox']) * 3).astype(int)

array([ 241, 1730,   98,  165])

In [36]:
result = {} 
for dat in data:
  if dat['score'] < 0.2:
    continue
  img = images_dict[dat['image_id']]['file_name'][:-4]
  if (img not in result):
    result[img] = ''
  result[img] += str(dat['category_id']) + ' ' + str(1.0) + ' '
  bbox = (np.array(dat['bbox']) * 3).astype(int) 
  bbox[2] += bbox[0]
  bbox[3] += bbox[1]
  for x in bbox:
    result[img] += str(x) + ' ' 
print(len(result))
for img__ in images_dict:
  img = img__['file_name'][:-4]
  
  if (img not in result):
    result[img] = '14 1.0 0 0 1 1'
  

2796


In [29]:
import pandas as pd 
df = pd.DataFrame() 
df['image_id'] = result.keys() 
df['PredictionString'] = result.values() 
df.to_csv('result.csv', index = False)

In [30]:
df.head()

,image_id,PredictionString
0,f2fd066f221a835b1c68b24dcbeddc14,0 1.0 1371 862 1724 1242
1,788c42c6b87bb62f1d5def6b0e7ca9da,0 1.0 1128 956 1416 1294
2,5d8d9b5c072bbc2a4a2a0b89cd35ee9f,0 1.0 1441 775 1773 1171
3,e541711f560cd700d7fdddd1d3dd98d4,0 1.0 946 684 1141 902
4,6ee6a322ef5a5c52b1e160c2ef5e83fd,0 1.0 1392 855 1688 1174 3 1.0 998 1592 2179 1...


In [31]:
for i, id in enumerate(result.keys()):
  print(id, result[id])
  if (i == 10):
    break

f2fd066f221a835b1c68b24dcbeddc14 0 1.0 1371 862 1724 1242 
788c42c6b87bb62f1d5def6b0e7ca9da 0 1.0 1128 956 1416 1294 
5d8d9b5c072bbc2a4a2a0b89cd35ee9f 0 1.0 1441 775 1773 1171 
e541711f560cd700d7fdddd1d3dd98d4 0 1.0 946 684 1141 902 
6ee6a322ef5a5c52b1e160c2ef5e83fd 0 1.0 1392 855 1688 1174 3 1.0 998 1592 2179 1989 
3075ea62bfa39a295cbf419b9248401b 0 1.0 1362 817 2006 1281 
6de2e33c4814358a4af07f0d2b03e2de 0 1.0 1423 854 1689 1111 
8b1be486cd3aa270873ee8bce40084a0 0 1.0 1103 749 1323 986 3 1.0 826 1387 1635 1652 
7ce84780a2d602c08c800ff2e769514c 3 1.0 1349 1399 2539 1898 
72cfa22b529d2ae71985b2636cb9baf8 0 1.0 1703 680 2010 1053 
fe52d8c7ce9e6f9435fcd3438aa19da7 3 1.0 876 1425 2099 1890 


In [35]:
for i,id in enumerate(images_dict):
  print(id)
  if (i == 10):
    break

{'license': 0, 'url': None, 'file_name': '973b316bee7e6ccda97c07d096008fea.jpg', 'height': 833, 'width': 682, 'date_captured': None, 'id': 0}
{'license': 0, 'url': None, 'file_name': '0eca5bbff12b841c18d231fc8cda0eeb.jpg', 'height': 833, 'width': 682, 'date_captured': None, 'id': 1}
{'license': 0, 'url': None, 'file_name': '8673c57e2f920c8bb15d3c8a062ee2a2.jpg', 'height': 914, 'width': 910, 'date_captured': None, 'id': 2}
{'license': 0, 'url': None, 'file_name': '28381c8323917ac88d571e1213a2d382.jpg', 'height': 1024, 'width': 1024, 'date_captured': None, 'id': 3}
{'license': 0, 'url': None, 'file_name': 'b0a1aba6962b7a2ae90ebd5f293a7413.jpg', 'height': 810, 'width': 664, 'date_captured': None, 'id': 4}
{'license': 0, 'url': None, 'file_name': '60485f64a1de6c652639c3ab84fcd12f.jpg', 'height': 833, 'width': 682, 'date_captured': None, 'id': 5}
{'license': 0, 'url': None, 'file_name': 'd343221fe29d689f6d6e5b1501774ee4.jpg', 'height': 1136, 'width': 885, 'date_captured': None, 'id': 6}
{'l